# Package loading

In [1]:
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation, add, concatenate

In [2]:
tf.version

<module 'tensorflow._api.v2.version' from 'C:\\OSGEO4~1\\apps\\Python37\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>

# Datasets Manipulations

In [3]:
import os
from glob import glob
import numpy as np
from osgeo import gdal
from matplotlib import pyplot as plt

In [4]:
path = u'../../'
width = 256
num = 250

In [65]:
def load_data(path, mode):
    images_path = glob(os.path.join(path, "tiles/*"))
    masks_path = glob(os.path.join(path, "masks/*"))
    if mode == 'train':
        image_path, mask_path = images_path[:25], masks_path[:25]
    elif mode == 'eval':
        image_path, mask_path = images_path[25:30], masks_path[25:30]
    else:
        image_path, mask_path = images_path[30:], masks_path[30:]
        
    return image_path, mask_path

In [66]:
image_path, mask_path = load_data(path, mode='train')
image_path, mask_path

(['../../tiles\\tile_WV3_Pansharpen_11_2016_1.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_10.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_11.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_12.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_13.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_14.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_15.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_16.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_17.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_18.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_19.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_2.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_20.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_21.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_22.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_23.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_24.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_25.tif',
  '../../tiles\\tile_WV3_Pansharpen_11_2016_26.t

In [6]:
def get_raster(raster_path):
    ds = gdal.Open(raster_path)
    data = np.empty((ds.RasterYSize, ds.RasterXSize, ds.RasterCount), dtype=np.float32)
    for b in range(1, ds.RasterCount + 1):
        band = ds.GetRasterBand(b).ReadAsArray()
        data[:, :, b-1] = band
    if data.shape[-1] > 1:
        data = norma_data(data, norma_methods='min-max')   
    return data

In [7]:
def norma_data(data, norma_methods="z-score"):
    arr = np.empty(data.shape, dtype=np.float32)
    for i in range(data.shape[-1]):
        array = data.transpose(2, 0, 1)[i, :, :]
        mins, maxs, mean, std= np.percentile(array, 1), np.percentile(array, 99), np.mean(array), np.std(array)
        if norma_methods == "z-score":
            new_array = (array-mean)/std
        else:
            new_array = np.clip(2*(array-mins)/(maxs-mins), 0, 1)
        arr[:, :, i] = new_array
    return arr

In [83]:
def pare_fun(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()
    
        x = get_raster(x)
        y = get_raster(y)
        return x, y
    
    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    image.set_shape([1000, 1000, 7])
    mask.set_shape([1000, 1000, 1])
    return image, mask

In [84]:
def dataset_generator(path, mode, width, num):
    images_path, masks_path = load_data(path, mode)
    datasets = tf.data.Dataset.from_tensor_slices((images_path, masks_path))
    ds = datasets.map(pare_fun)
    for images, masks in ds:
        for i in range(num):
            location = np.random.randint(width/2 - 1, 999 - width/2, (2,))
            h, w = location[0], location[1]
            d1 = int(width/2 - 1)
            d2 = int(width - d1)
            image_patch = np.expand_dims(images[h-d1: h+d2, w-d1: w+d2, :], axis=0)
            mask_patch = np.expand_dims(masks[h-d1: h+d2, w-d1: w+d2], axis=0)
            yield image_patch, mask_patch

In [85]:
train_dataset_generator = dataset_generator(path=path, mode='train', width=width, num=num)
eval_dataset_generator = dataset_generator(path=path, mode='eval', width=width, num=num)
test_dataset_generator = dataset_generator(path=path, mode='test', width=width, num=num)

In [82]:
# len(train_dataset_generator)

In [74]:
for image_patch, mask_patch in train_dataset_generator:
    print(image_patch.shape, mask_patch.shape)

(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 

(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 

(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 

(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 

(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 

...

(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 

(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 

(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 

(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 

(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 7) (1, 256, 256, 1)
(1, 256, 256, 

In [ ]:
# plt.imshow(first_sample['mask'][:, :])
# plt.show()

In [ ]:
# plt.imshow(first_sample['image'][:, :, 0:3])
# plt.show()

# Model Construction

In [16]:
def res_block(x, nb_filters, strides):
    res_path = BatchNormalization()(x)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same', strides=strides[0])(res_path)
    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same', strides=strides[1])(res_path)

    shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1), strides=strides[0])(x)
    shortcut = BatchNormalization()(shortcut)

    res_path = add([shortcut, res_path])
    return res_path

In [17]:
def encoder(x):
    to_decoder = []

    main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1))(x)
    main_path = BatchNormalization()(main_path)
    main_path = Activation(activation='relu')(main_path)

    main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1))(main_path)

    shortcut = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1))(x)
    shortcut = BatchNormalization()(shortcut)

    main_path = add([shortcut, main_path])
    # first branching to decoder
    to_decoder.append(main_path)

    main_path = res_block(main_path, [128, 128], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    main_path = res_block(main_path, [256, 256], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    return to_decoder

In [18]:
def decoder(x, from_encoder):
    main_path = UpSampling2D(size=(2, 2))(x)
    main_path = concatenate([main_path, from_encoder[2]], axis=3)
    main_path = res_block(main_path, [256, 256], [(1, 1), (1, 1)])

    main_path = UpSampling2D(size=(2, 2))(main_path)
    main_path = concatenate([main_path, from_encoder[1]], axis=3)
    main_path = res_block(main_path, [128, 128], [(1, 1), (1, 1)])

    main_path = UpSampling2D(size=(2, 2))(main_path)
    main_path = concatenate([main_path, from_encoder[0]], axis=3)
    main_path = res_block(main_path, [64, 64], [(1, 1), (1, 1)])

    return main_path

In [19]:
def build_res_unet(input_shape):
    inputs = Input(shape=input_shape)

    to_decoder = encoder(inputs)

    path = res_block(to_decoder[2], [512, 512], [(2, 2), (1, 1)])

    path = decoder(path, from_encoder=to_decoder)

    path = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(path)

    return Model(inputs=inputs, outputs=path)

In [20]:
input_shape = (width, width, 7)

In [21]:
model = build_res_unet(input_shape=input_shape)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 7) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 4096        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 512         input_1[0][0]                    
_______________________________________________________________________________________

# Model Compile

In [ ]:
# Optimizer, Loss and metrics
# Model.compile()

In [22]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [54]:
def dice(y_true, y_pred): 
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / denominator

In [23]:
def dice_loss(y_true, y_pred): 
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - (numerator / denominator)

In [24]:
def cedice_loss(y_true, y_pred):
    def dice_loss(y_true, y_pred):
        y_pred = tf.math.sigmoid(y_pred)
        numerator = 2 * tf.reduce_sum(y_true * y_pred)
        denominator = tf.reduce_sum(y_true + y_pred)
        return 1 - numerator / denominator
    y_true = tf.cast(y_true, tf.float32)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred) + dice_loss(y_true, y_pred)
    return tf.reduce_mean(loss)

In [90]:
# y_pred = np.array([0.5, 0.6, 0.2, 0.1], dtype=np.float32)
# y_true = np.array([1, 1, 0, 0], dtype=np.float32)

In [91]:
# y_pred = y_pred.reshape((2, 2))
# y_true = y_true.reshape((2, 2))

In [ ]:
# a = tf.math.sigmoid(y_pred)
# a

In [ ]:
# o = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred)
# o

In [ ]:
# loss = cedice_loss(y_true, tf.math.log(y_pred / (1 - y_pred)))
# loss

In [ ]:
# loss1 = cedice_loss(y_true, y_pred)
# loss1

In [ ]:
# dice_loss = dice_loss(y_true, y_pred)
# dice_loss

In [55]:
model.compile(loss=dice_loss, optimizer=optimizer, metrics=[dice])

# Model Training

In [89]:
model.fit(train_dataset_generator, batch_size=15, epochs=10, validation_data=eval_dataset_generator,validation_batch_size=50)

Epoch 1/10
     28/Unknown - 72s 3s/step - loss: 0.3238 - dice: 0.8606

KeyboardInterrupt: 

In [87]:
# model.fit?

In [ ]:
model.save('model')